<a href="https://colab.research.google.com/github/Sooryakiran/HARTS/blob/master/harts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hardware Architectural Search (HARTS)
--------------------------------------

DISCLAIMER: This is not a binary neural network where the weights are zeros and ones.

This notebook illustrates the use of ideas from deep learning to design hardwares for predictive applications. The below code can generate architectures to classify MNIST handwritten digits using just logic gates.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import numpy as np

# Choice of gates between any two values

Here I have implimented 3 possible choices of gates, AND, OR and XOR. The choice has to be differentiable. So I used a weight value for each gate over which softmax will be taken to choose the gate. Since softmax is differentiable, the choice of gates become differentiable and we can use backpropagation to tune the weights to choose the gate that gives better results.


 1 represents on state
 
-1 represents off state

In [67]:
class BinLayer(nn.Module):
    """
    Class BinLayer

    """
    def __init__(self, in_neurons, temp = 0.01):
        """
        The class constructor

        @param in_neurons : int, bit size of input
        @param temp       : float, temperature of softmax

        Let us say that there are n bits as input. We have n x n possible input pairs.
        However this reduces by nearly a factor of 2 for commutative bitwise operations.
        In this module, we take the entire n x n as search domain, so that we can support
        bitwise operations that are not commutative in future.

        Assume we have an input bit vector x. We compute 3 output matrices,
        OR  (x', x) 
        AND (x', x)
        XOR (x', x)
        Where x' is the transpose of x.

        During forward pass, we choose one among theses 3 matrices element wise to
        create an nxn matrix. The choice is done by doing a softmax over the stored weights.
        The weight matrix is updated using backpropagation to improve the choice.

        """

        super(BinLayer, self).__init__()
        self.in_neurons = in_neurons

        self.and_ = AND()
        self.or_  = OR()
        self.xor_ = XOR()
        
        self.weights = torch.nn.Parameter(data = torch.Tensor(in_neurons*in_neurons, 3), requires_grad = True)
        self.softmax = nn.Softmax(dim = -1)
        self.temp    = temp

        self.weights.data.uniform_(-1, 1)

    def forward(self, x):
        """
        The forward pass

        @param x: input bit vector

        """
        and_output = self.and_(x).unsqueeze(-1)
        or_output  = self.or_(x).unsqueeze(-1)
        xor_output = self.xor_(x).unsqueeze(-1)

        soft_weights   = self.softmax(self.weights/self.temp)
        massive_input  = torch.cat([and_output, or_output, xor_output], -1)
        massive_weight = soft_weights.unsqueeze(0).repeat(x.size(0), 1, 1)
        massive_output = torch.mul(massive_input, massive_weight)

        return torch.sum(massive_output, axis = -1)

class SELECT(nn.Module):
    """
    The SELECT class

    """
    def __init__(self, in_neurons, out_neurons, temp = 0.01):
        """
        Since the BinLayer defined above is not scalable, i.e it produces nxn outputs
        for an input of size n, we define a module that subsamples the input. The sub
        sampling choices are also trainable using backpropagation, since we use 
        softmax over the stored weights to perform the choice.

        @param in_neurons  : int, input bit vector length
        @param out_neurons : int, subsampled output bit vector length

        """
        super(SELECT, self).__init__()
        self.weights = torch.nn.Parameter(data = torch.Tensor(in_neurons, out_neurons), requires_grad = True)
        self.softmax = nn.Softmax(dim = 0)
        self.temp    = temp

        self.weights.data.uniform_(-1, 1)

    def forward(self, x):
        """
        Selects the most suitable top out_neurons

        """
        soft_weights = self.softmax(self.weights/self.temp)
        return x @ soft_weights

class NotLayer(nn.Module):
    """
    The NOT layer

    """
    def __init__(self, in_neurons, temp = 0.01):
        """
        The NOT layer can be used as a possible 'activation function'. This layer 
        chooses whether to put a NOT gate or not on each elements in the input bit
        vector. Like above, the choice is differentiable.

        @param in_neurons : int, input bit vector length
        @param temp       : float, softmax temperature

        """
        super(NotLayer, self).__init__()
        self.weights = torch.nn.Parameter(data = torch.Tensor(in_neurons, 2), requires_grad = True)
        self.softmax = nn.softmax(dim = -1)
        self.temp    = temp

        self.weights.data.uniform_(-1, 1)

    def forward(self, x):
        """
        The forward pass

        """
        x_in            = x.unsqueeze(0)
        x_compliment_in = -x_in
        soft_weights    = self.softmax(self.weights/self.temp)
        massive_inputs  = torch.cat([x_in, x_compliment_in], -1)
        massive_weights = soft_weights.unsqueeze(0).repeat(x.size(0), 1)
        massive_output  = torch.mul(massive_input, massive_weight)

        return torch.sum(massive_output, axis = -1)
        
class XOR(nn.Module):
    """
    The XOR module

    """
    def __init__(self):
        super(XOR, self).__init__()
    
    def forward(self, x):
        x      = x.unsqueeze(-1)
        output = -x @ torch.transpose(x, 1, 2)

        return output.view(x.size(0), -1)


class AND(nn.Module):
    """
    The AND module

    """
    def __init__(self):
        super(AND, self).__init__()
        self.xor = XOR();

    def forward(self, x):
        x = (x+1)/2 
        return - self.xor(x)*2 -1

class OR(nn.Module):
    """
    The OR module
    
    """
    def __init__(self):
        super(OR, self).__init__()
        self.xor  =  XOR()
        self.and_ = AND();

    def forward(self, x):
        return self.xor(x) + self.and_(x) + 1

In [68]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()

        self.select_1 = SELECT(in_neurons = 784, out_neurons = 256)
        self.bin_1    = BinLayer(in_neurons = 256)

        self.select_2 = SELECT(in_neurons = 256*256, out_neurons = 256)
        self.bin_2    = BinLayer(in_neurons = 256)

        self.select_3 = SELECT(in_neurons = 256*256, out_neurons = 128)
        self.bin_3    = BinLayer(in_neurons = 128)

        self.select_4 = SELECT(in_neurons = 128*128, out_neurons= 64)
        self.bin_4    = BinLayer(in_neurons = 64)

        self.select_5 = SELECT(in_neurons = 64*64, out_neurons = 32)
        self.bin_5    = BinLayer(in_neurons = 32)

        self.select_6 = SELECT(in_neurons = 32*32, out_neurons = 10)

    def forward(self, x):
        x = self.select_1(x)
        x = self.bin_1(x)
        x = self.select_2(x)
        x = self.bin_2(x)
        x = self.select_3(x)
        x = self.bin_3(x)
        x = self.select_4(x)
        x = self.bin_4(x)
        x = self.select_5(x)
        x = self.bin_5(x)
        x = self.select_6(x)
        return x

# Hyperparameters

In [36]:
EPOCHS = 10
LEARNING_RATES = [1e-2, 1e-3]
MOMENTUM = 0.5
BATCH_SIZE = 512
LOG_INTERVAL = 5

In [37]:
class CustomTransform:
    def __init__(self):
        pass
    def __call__(self, x):
        x = x*2 - 1
        return x.view(-1)

TRAIN_LOADER = torch.utils.data.DataLoader(torchvision.datasets.MNIST('/files/', train=True, download=True,
                                           transform=torchvision.transforms.Compose([
                                           torchvision.transforms.ToTensor(),
                                           CustomTransform()])),
                                           batch_size=BATCH_SIZE, shuffle=True)

TEST_LOADER = torch.utils.data.DataLoader(torchvision.datasets.MNIST('/files/', train=False, download=True,
                                          transform=torchvision.transforms.Compose([
                                          torchvision.transforms.ToTensor(),
                                          CustomTransform()])),
                                          batch_size=BATCH_SIZE, shuffle=True)

In [73]:
def train(network, epoch):
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda"
        print("TRAINING ON GPU")

    network = network.to(device)
    criterion = nn.CrossEntropyLoss()
    if epoch < len(LEARNING_RATES):
        lr = LEARNING_RATES[epoch]
    else:
        lr = LEARNING_RATES[-1]
    optimizer = torch.optim.Adam(network.parameters(), lr=lr)
    for index, (data, target) in enumerate(TRAIN_LOADER):
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = network(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if index%LOG_INTERVAL == 0:
            pred = output.data.max(1, keepdim=True)[1]
            acc = pred.eq(target.data.view_as(pred)).sum()*1.0/BATCH_SIZE
        
            print("EPOCH: %d: BATCH (%d), \tLOSS = %0.3f, \tACC:%0.2f" %(epoch, index, loss.item(), acc.item()))

def test(network):
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda"
        print("TESTING ON GPU")

    network = network.to(device)
    criterion = nn.CrossEntropyLoss()
    

    for index, (data, target) in enumerate(TEST_LOADER):
        data = data.to(device)
        target = target.to(device)
        output = network(data)
        loss = criterion(output, target)
        loss.backward()

        if index%LOG_INTERVAL == 0:
            pred = output.data.max(1, keepdim=True)[1]
            acc = pred.eq(target.data.view_as(pred)).sum()*1.0/BATCH_SIZE
        
            print("BATCH (%d), \tLOSS = %0.3f, \tACC:%0.2f" %(index, loss.item(), acc.item()))


def train_multiple_epochs(network):
    for epoch in range(EPOCHS):
        train(network, epoch)


In [39]:
network = Network()
train_multiple_epochs(network)

TRAINING ON GPU
EPOCH: 0: BATCH (0), 	LOSS = 2.303, 	ACC:0.11
EPOCH: 0: BATCH (5), 	LOSS = 2.123, 	ACC:0.19
EPOCH: 0: BATCH (10), 	LOSS = 2.046, 	ACC:0.24
EPOCH: 0: BATCH (15), 	LOSS = 1.909, 	ACC:0.27
EPOCH: 0: BATCH (20), 	LOSS = 1.782, 	ACC:0.40
EPOCH: 0: BATCH (25), 	LOSS = 1.711, 	ACC:0.41
EPOCH: 0: BATCH (30), 	LOSS = 1.695, 	ACC:0.42
EPOCH: 0: BATCH (35), 	LOSS = 1.680, 	ACC:0.44
EPOCH: 0: BATCH (40), 	LOSS = 1.625, 	ACC:0.47
EPOCH: 0: BATCH (45), 	LOSS = 1.625, 	ACC:0.43
EPOCH: 0: BATCH (50), 	LOSS = 1.645, 	ACC:0.46
EPOCH: 0: BATCH (55), 	LOSS = 1.612, 	ACC:0.46
EPOCH: 0: BATCH (60), 	LOSS = 1.637, 	ACC:0.49
EPOCH: 0: BATCH (65), 	LOSS = 1.626, 	ACC:0.48
EPOCH: 0: BATCH (70), 	LOSS = 1.636, 	ACC:0.49
EPOCH: 0: BATCH (75), 	LOSS = 1.661, 	ACC:0.42
EPOCH: 0: BATCH (80), 	LOSS = 1.603, 	ACC:0.49
EPOCH: 0: BATCH (85), 	LOSS = 1.606, 	ACC:0.47
EPOCH: 0: BATCH (90), 	LOSS = 1.620, 	ACC:0.49
EPOCH: 0: BATCH (95), 	LOSS = 1.627, 	ACC:0.48
EPOCH: 0: BATCH (100), 	LOSS = 1.600, 	ACC:0.4

In [40]:
torch.save(network, "saved_model.pth")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Network. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type SELECT. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type BinLayer. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type AND. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-pa

In [74]:
test(network)

TESTING ON GPU
BATCH (0), 	LOSS = 1.428, 	ACC:0.60
BATCH (5), 	LOSS = 1.479, 	ACC:0.56
BATCH (10), 	LOSS = 1.495, 	ACC:0.54
BATCH (15), 	LOSS = 1.469, 	ACC:0.58


# Check the confidence of each choice so that we can freeze and convert them into gates.

In [61]:
for name, param in network.named_parameters():
    if param.requires_grad:
        if "select" in name:
            print("SELECT")
            softmax = nn.Softmax(dim = 0)
            weights = param.data / 0.01
            outs = torch.transpose(softmax(weights), 0, 1)
            outs, _ = torch.max(outs, dim = -1)
            print(torch.mean(outs))
            
        elif "bin" in name:
            print("BIN")
            softmax = nn.Softmax(dim = -1)
            weights = param.data / 0.01
            outs, _ = torch.max(softmax(weights), dim = -1)
            print(torch.mean(outs))
        else:
            print("UNKNOWN")
        #print(name, param.data)

SELECT
tensor(0.9628, device='cuda:0')
BIN
tensor(0.9976, device='cuda:0')
SELECT
tensor(0.9663, device='cuda:0')
BIN
tensor(0.9983, device='cuda:0')
SELECT
tensor(0.9578, device='cuda:0')
BIN
tensor(0.9974, device='cuda:0')
SELECT
tensor(0.9428, device='cuda:0')
BIN
tensor(0.9969, device='cuda:0')
SELECT
tensor(0.9087, device='cuda:0')
BIN
tensor(0.9949, device='cuda:0')
SELECT
tensor(1., device='cuda:0')
